In [1]:
class Operator:
    def __init__(self,unique_id,children,time_required,is_join=False):
#         print ("initialized operator {}".format(unique_id))
        self.children = children
        self.log = []
        self.unique_id = unique_id
        self.time_required = time_required
        self.is_join = is_join
        self.paused_job = None
        
class Lattice:
    def __init__(self, operators):
        self.operators = operators
        self.children_dir = {}
        for op in operators:
            self.children_dir[op.unique_id] = op.children 
        # should prolly sort the operators 
    def __repr__(self):
        return "Lattice {}".format(self.children_dir.__repr__())            
    def get_children(self,operator):
        return self.children_dir[operator]
    def add_next(self,event,new_event_queue,curr_time):
        '''
            Adds new events to queue if necessary when an Event completes
        '''
        children = self.get_children(event.operator)
        for c in children:
            c = self.get_op(c)
            event_id = event.unique_id
            if c.is_join:
                if not c.paused_job:
                    c.paused_job = event
                    continue
                else:
                    event_id = min(event.unique_id,c.paused_job.unique_id)
                    c.paused_job = None
            new_event_queue.append(Event(event_id,c.unique_id, curr_time))
    def get_op(self,op):
        '''
            Assumes that the operators are in sorted and consecutive order
        '''
        out = self.operators[op]
        if out.unique_id != op:
            print ("ERROR: lattice not properly ordered [{} returned instead of {}]".format(out.unique_id,op))
        return out



In [2]:
op4 = Operator(4,[],8)
op3 = Operator(3,[4],6,is_join=True)
op2 = Operator(2,[3],4)
op1 = Operator(1,[3],2)
op0 = Operator(0,[1,2],10)

lat = Lattice([op0,op1,op2,op3,op4])

print(lat)
print (lat.get_children(3))

# Lattice {0: [1, 2], 1: [3], 2: [3], 3: [4], 4: []}
# [4]

Lattice {0: [1, 2], 1: [3], 2: [3], 3: [4], 4: []}
[4]


``` 
v1 -> v3 -> v4
^      ^   
|      |
v0 ->  v2
```

In [3]:
            
class Event:
    def __init__(self,  unique_id, operator , arrival_time):
        self.unique_id = unique_id
        self.operator = operator
        self.arrival_time = arrival_time
        self.time_remaining = -1
        self.start_time = None
        self.finish_time = None
    def __repr__(self):
        return "<Event {}; Running Op {}; Available at Time {}; Executed {} to {}>".format(self.unique_id,self.operator, self.arrival_time, self.start_time, self.finish_time)
    def start(self,lattice,time):
        self.time_remaining = lattice.get_op(self.operator).time_required
        self.start_time = time
    def step(self):
        if self.time_remaining < 1:
            print("ERROR: already finished event; {} left".format(self.time_remaining))
        self.time_remaining -= 1
        return self.time_remaining
    def finish(self,new_event_queue,lattice,time):
        if self.time_remaining != 0:
            print("ERROR: try to finish but not done; {} left".format(self.time_remaining))
        lattice.add_next(self,new_event_queue,time)
        self.finish_time = time + 1 # cause you technically am using up the current time slice
        
        
        
        


In [4]:
event_queue = []
e = Event(1,0,0)
e.start(lat,0)
for i in range(10):
    e.step()
e.finish(event_queue,lat,2)
e,event_queue
# (<Event 1; Running Op 0; Available at Time 0; Executed 0 to 2>,
#  [<Event 1; Running Op 1; Available at Time 2; Executed None to None>,
#   <Event 1; Running Op 2; Available at Time 2; Executed None to None>])


(<Event 1; Running Op 0; Available at Time 0; Executed 0 to 3>,
 [<Event 1; Running Op 1; Available at Time 2; Executed None to None>,
  <Event 1; Running Op 2; Available at Time 2; Executed None to None>])

In [5]:



class Worker: 
    def __init__(self, unique_id, gpus=1):
        self.unique_id = unique_id
        self.history = []
        self.current_event = None
    def __repr__(self):
        return "Worker {}: {}".format(self.unique_id, self.history)
    def do_job(self, task, lattice,time):
        if self.current_event != None:
            print("ERROR: Worker is still busy")
        self.history.append(task)
        self.current_event = task
        task.start(lattice,time)
    def reset(self):
        self.__init__(self.unique_id)
        
class WorkerPool:
    def __init__(self, workers):
        self.workers = workers
        self.count = len(workers)
    def __repr__(self):
        output = ""
        for w in self.workers:
            output += "\n{}".format(w)
        return "Worker Pool: " + output
    def add_worker(self,w):
        for v in self.workers:
            if v.unique_id == w.unique_id:
                print ("ERROR: worker not unique [{}]".format(w.unique_id))
                return 
        self.workers.append(w)
    def add_workers(self,ws):
        for w in ws:
            self.add_worker(w)
    def reset(self):
        for w in self.workers:
            w.reset()
        
            
    

In [6]:
w = Worker(1)
w.do_job(Event(7,4,2),lat,2)
w # Worker 1: [<Event 7; Running Op 4; Available at Time 2>]

Worker 1: [<Event 7; Running Op 4; Available at Time 2; Executed 2 to None>]

In [7]:
wp = WorkerPool([])
v = Worker(1)
wp.add_workers([w,v])
# ERROR: worker not unique [1]

wp
# Worker Pool: 
# Worker 1: [<Event 7; Running Op 4; Available at Time 2>]

ERROR: worker not unique [1]


Worker Pool: 
Worker 1: [<Event 7; Running Op 4; Available at Time 2; Executed 2 to None>]

In [8]:
def simulate(schedule,task_set,worker_pool,lattice,timeout):
    if not task_set:
        return
    task_set.sort(key=(lambda e: e.arrival_time))
    event_queue = []
    for time in range(timeout):
        print ("step: {}".format(time))
        while len(task_set) != 0 and  task_set[0].arrival_time == time:
            task = task_set.pop(0)
            print ("Activate: {}".format(task))
            event_queue.append(task)
        schedule(time,event_queue,lattice,worker_pool)
def fifo_schedule(time,event_queue,lattice,worker_pool):
    new_event_queue = []
    print (event_queue)
    for worker in worker_pool.workers:
        if worker.current_event == None and event_queue:
            worker.do_job(event_queue.pop(0),lattice,time)
        if worker.current_event: 
            worker.current_event.step()
            if worker.current_event.time_remaining == 0:
                worker.current_event.finish(new_event_queue,lattice,time)
                print("finished event: {}".format(worker.current_event))
                worker.current_event = None
    event_queue.extend(new_event_queue)

            


In [9]:
wp = WorkerPool([])
wp.add_worker(v)
wp

Worker Pool: 
Worker 1: []

In [10]:

simulate(fifo_schedule,[Event(2,0,4),Event(5,0,40)],wp,lat,100)

step: 0
[]
step: 1
[]
step: 2
[]
step: 3
[]
step: 4
Activate: <Event 2; Running Op 0; Available at Time 4; Executed None to None>
[<Event 2; Running Op 0; Available at Time 4; Executed None to None>]
step: 5
[]
step: 6
[]
step: 7
[]
step: 8
[]
step: 9
[]
step: 10
[]
step: 11
[]
step: 12
[]
step: 13
[]
finished event: <Event 2; Running Op 0; Available at Time 4; Executed 4 to 14>
step: 14
[<Event 2; Running Op 1; Available at Time 13; Executed None to None>, <Event 2; Running Op 2; Available at Time 13; Executed None to None>]
step: 15
[<Event 2; Running Op 2; Available at Time 13; Executed None to None>]
finished event: <Event 2; Running Op 1; Available at Time 13; Executed 14 to 16>
step: 16
[<Event 2; Running Op 2; Available at Time 13; Executed None to None>]
step: 17
[]
step: 18
[]
step: 19
[]
finished event: <Event 2; Running Op 2; Available at Time 13; Executed 16 to 20>
step: 20
[<Event 2; Running Op 3; Available at Time 19; Executed None to None>]
step: 21
[]
step: 22
[]
step: 

In [11]:
v.history

[<Event 2; Running Op 0; Available at Time 4; Executed 4 to 14>,
 <Event 2; Running Op 1; Available at Time 13; Executed 14 to 16>,
 <Event 2; Running Op 2; Available at Time 13; Executed 16 to 20>,
 <Event 2; Running Op 3; Available at Time 19; Executed 20 to 26>,
 <Event 2; Running Op 4; Available at Time 25; Executed 26 to 34>,
 <Event 5; Running Op 0; Available at Time 40; Executed 40 to 50>,
 <Event 5; Running Op 1; Available at Time 49; Executed 50 to 52>,
 <Event 5; Running Op 2; Available at Time 49; Executed 52 to 56>,
 <Event 5; Running Op 3; Available at Time 55; Executed 56 to 62>,
 <Event 5; Running Op 4; Available at Time 61; Executed 62 to 70>]

In [12]:
wp.reset()
v.history

[]

In [13]:
x = Worker(2)
x.reset()
wp.add_worker(x)
wp

Worker Pool: 
Worker 1: []
Worker 2: []

In [14]:
simulate(fifo_schedule,[Event(2,0,4)],wp,lat,100)

step: 0
[]
step: 1
[]
step: 2
[]
step: 3
[]
step: 4
Activate: <Event 2; Running Op 0; Available at Time 4; Executed None to None>
[<Event 2; Running Op 0; Available at Time 4; Executed None to None>]
step: 5
[]
step: 6
[]
step: 7
[]
step: 8
[]
step: 9
[]
step: 10
[]
step: 11
[]
step: 12
[]
step: 13
[]
finished event: <Event 2; Running Op 0; Available at Time 4; Executed 4 to 14>
step: 14
[<Event 2; Running Op 1; Available at Time 13; Executed None to None>, <Event 2; Running Op 2; Available at Time 13; Executed None to None>]
step: 15
[]
finished event: <Event 2; Running Op 1; Available at Time 13; Executed 14 to 16>
step: 16
[]
step: 17
[]
finished event: <Event 2; Running Op 2; Available at Time 13; Executed 14 to 18>
step: 18
[<Event 2; Running Op 3; Available at Time 17; Executed None to None>]
step: 19
[]
step: 20
[]
step: 21
[]
step: 22
[]
step: 23
[]
finished event: <Event 2; Running Op 3; Available at Time 17; Executed 18 to 24>
step: 24
[<Event 2; Running Op 4; Available at Ti

In [15]:
x.history

[<Event 2; Running Op 2; Available at Time 13; Executed 14 to 18>]

In [16]:
v.history

[<Event 2; Running Op 0; Available at Time 4; Executed 4 to 14>,
 <Event 2; Running Op 1; Available at Time 13; Executed 14 to 16>,
 <Event 2; Running Op 3; Available at Time 17; Executed 18 to 24>,
 <Event 2; Running Op 4; Available at Time 23; Executed 24 to 32>]